# Bot для торговли на бирже Binance

https://github.com/binance/binance-futures-connector-python

Стратегия торговли: вход в сделку на основе предсказаний модели обученной на данных стакана.

## Импорт библиотек

In [32]:
from binance.client import Client
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.lib.utils import config_logging
import pandas as pd
import numpy as np
import talib
import keys
import time
from datetime import datetime
from pytz import timezone
import logging
import requests
import json
import re
pd.options.mode.chained_assignment = None # отключение предупреждения
pd.set_option('display.max_columns', None) # показывать максимальное количесвто столбцов

## Подключение к бирже

In [93]:
client = UMFutures(keys.api_key, keys.secret_key)

## Задать переменные

In [94]:
SYMBOL = 'MOODENGUSDT'

## Запуск логирования

In [95]:
def start_logging(Bot_name):
    """
    Bot_name - имя бота - str
    """
    logger = logging.getLogger(Bot_name)
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler('bot.log')
    fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(fh)
    return logger

## Получение данных

### REST_API

#### Стакан

In [96]:
client.depth(SYMBOL, limit = 10)

{'lastUpdateId': 7920350983101,
 'E': 1751359799694,
 'T': 1751359799684,
 'bids': [['0.1403700', '8415'],
  ['0.1403600', '10481'],
  ['0.1403500', '15864'],
  ['0.1403400', '31850'],
  ['0.1403300', '23027'],
  ['0.1403200', '31258'],
  ['0.1403100', '52285'],
  ['0.1403000', '27753'],
  ['0.1402900', '27419'],
  ['0.1402800', '43518']],
 'asks': [['0.1403800', '1957'],
  ['0.1403900', '1507'],
  ['0.1404000', '4727'],
  ['0.1404100', '2091'],
  ['0.1404200', '4153'],
  ['0.1404300', '9567'],
  ['0.1404400', '23883'],
  ['0.1404500', '19609'],
  ['0.1404600', '35198'],
  ['0.1404700', '13163']]}

In [97]:
df = pd.DataFrame(client.depth(SYMBOL, limit = 10))

In [98]:
df.head(10)

,lastUpdateId,E,T,bids,asks
0,7920351360800,1751359802012,1751359802001,"[0.1404500, 8499]","[0.1404600, 76]"
1,7920351360800,1751359802012,1751359802001,"[0.1404400, 15859]","[0.1404700, 194]"
2,7920351360800,1751359802012,1751359802001,"[0.1404300, 14872]","[0.1404800, 2715]"
3,7920351360800,1751359802012,1751359802001,"[0.1404200, 21184]","[0.1404900, 5175]"
4,7920351360800,1751359802012,1751359802001,"[0.1404100, 24514]","[0.1405000, 6850]"
5,7920351360800,1751359802012,1751359802001,"[0.1404000, 39574]","[0.1405100, 37131]"
6,7920351360800,1751359802012,1751359802001,"[0.1403900, 23587]","[0.1405200, 33304]"
7,7920351360800,1751359802012,1751359802001,"[0.1403800, 44468]","[0.1405300, 19134]"
8,7920351360800,1751359802012,1751359802001,"[0.1403700, 46930]","[0.1405400, 14299]"
9,7920351360800,1751359802012,1751359802001,"[0.1403600, 40544]","[0.1405500, 41617]"


### WebSockets

#### Функции для подключения

In [86]:
def start_Socket(handler):
    '''
    Запуск сокета
    '''
    my_client = UMFuturesWebsocketClient(on_message = handler)
    try:
        my_client.diff_book_depth(
        symbol="moodengusdt",
        speed=500,
        id=1)
        time.sleep(10)
        logging.debug("closing ws connection")
        my_client.stop()
        
    except:
        time.sleep(2)
        logging.debug("connection restart")
        my_client.diff_book_depth(
        symbol="moodengusdt",
        speed=150,
        id=1)
        time.sleep(15000)
        logging.debug("closing ws connection")
        my_client.stop()


In [79]:
data = []

In [80]:
def handler_debug(_, message):
    """
    Вывод всей инф стакана
    """
    if json.loads(message)['e'] == 'depthUpdate':
        data.append(message)
        print(message)

In [81]:
def handler_orderbook_update(_, message):
    """
    Предобработка данных стакана
    """
    if json.loads(message)['e'] == 'depthUpdate':
        try:
            bids = [float(b[1]) for b in json.loads(message)['b']][::-1][:10]
            asks = [float(a[1]) for a in json.loads(message)['a']][:10]
            mid_price = [(float(json.loads(message)['a'][-1][0]) + float(json.loads(message)['b'][::-1][0][0])) / 2] 
            total_feat = mid_price + asks + bids 
            df_row = pd.DataFrame(total_feat).T
            df_row.columns = ['mid_price', 'ask_1', 'ask_2', 'ask_3', 'ask_4', 'ask_5', 'ask_6', 'ask_7', 'ask_8', 'ask_9', 'ask_10', 
                        'bid_1', 'bid_2', 'bid_3', 'bid_4', 'bid_5', 'bid_6', 'bid_7', 'bid_8', 'bid_9', 'bid_10']
            data.append(df_row)
            #print(message)
        except:
            df_row = pd.DataFrame()  
            data.append(df_row) 

#### Запуск сбора данных

In [87]:
start_Socket(handler_orderbook_update)

ERROR:binance.websocket.websocket_client:Error from callback <function handler_orderbook_update at 0x000002348B9C3010>: 'e'


#### Получение датафрейма

In [88]:
ds_ws = pd.concat(data).reset_index(drop=True)

In [89]:
ds_ws.shape

(14, 21)

In [90]:
ds_ws.head(2)

,mid_price,ask_1,ask_2,ask_3,ask_4,ask_5,ask_6,ask_7,ask_8,ask_9,ask_10,bid_1,bid_2,bid_3,bid_4,bid_5,bid_6,bid_7,bid_8,bid_9,bid_10
0,0.14496,7802.0,7136.0,3991.0,11035.0,12301.0,13767.0,10815.0,18280.0,13433.0,12602.0,0.0,317.0,5354.0,16992.0,19571.0,29983.0,29873.0,40955.0,12106.0,29851.0
1,0.14065,0.0,0.0,118.0,999.0,7142.0,6867.0,11901.0,10979.0,13071.0,8835.0,4405.0,11702.0,10563.0,16798.0,20541.0,23456.0,29225.0,34678.0,33748.0,15981.0


#### Выгрузка датафрейма

In [91]:
ds_ws.to_csv('ds_moodeng_13.csv', index = False)